In [1]:
# 라이브러리 설치 및 임포트
!pip install transformers evaluate sacrebleu pandas scikit-learn -q

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
from sklearn.model_selection import train_test_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.3 MB/s eta 0:00:00


In [2]:
# 모델 및 토크나이저 로드
model_path = "pilan2/cai-translator"

print(f"Loading model from {model_path}...")
try:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    # GPU가 있다면 사용
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    print(f"Model loaded successfully on {device}!")
except Exception as e:
    print(f"Error loading model: {e}")

Loading model from pilan2/cai-translator...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/849 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/842k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/310M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

Model loaded successfully on cpu!


In [3]:
# 데이터 로드 및 Test Set 분리
df = pd.read_csv('data.csv')

# 데이터 분할 (Train 80% / Val 10% / Test 10%)
SEED = 42
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=SEED)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=SEED)

print(f"Total Data: {len(df)}")
print(f"Test Set Size: {len(test_df)}")

Total Data: 100
Test Set Size: 10


In [4]:
# 평가 지표(BLEU) 로드
bleu = evaluate.load("sacrebleu")

def generate_predictions(batch_texts, model, tokenizer, device):
    # 입력 텍스트 토크나이징
    inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=128)

    # 데이터를 GPU로 이동
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # 모델 추론 (Beam Search 사용)
    outputs = model.generate(
        **inputs,
        max_length=128,
        num_beams=5,
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    # 디코딩
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [5]:
# 실제 평가 수행
print("\nGenerating predictions for Test Set...")

# 입력 및 정답
test_inputs = test_df['intent'].tolist()
test_references = test_df['my_trans'].tolist()

# 예측 생성
predictions = generate_predictions(test_inputs, model, tokenizer, device)


Generating predictions for Test Set...


In [6]:
# 결과 계산 및 출력
results = bleu.compute(predictions=predictions, references=test_references)

print("\n" + "="*30)
print(f" >>> Final BLEU Score: {results['score']:.2f}")
print("="*30 + "\n")

# 샘플 출력 (실제 번역 결과 눈으로 확인)
print("=== Sample Predictions ===")
for i in range(len(test_df)):
    print(f"[Input (KR)]: {test_inputs[i]}")
    print(f"[Target (EN)]: {test_references[i]}")
    print(f"[Model  (EN)]: {predictions[i]}")
    print("-" * 50)


 >>> Final BLEU Score: 49.77

=== Sample Predictions ===
[Input (KR)]: 좋아. 따라갈게.

미소 짓고 그와 함께 걷기 시작한다
[Target (EN)]: Cool. I'll follow you.

*smile, then start walk with him*
[Model  (EN)]: Okay. I'll follow you. *smile, then start walking with him*
--------------------------------------------------
[Input (KR)]: 잠시 놀란 표정을 짓고 다시 미소 짓는다

... 나도.

그리고 그에게 살짝 기댄다
[Target (EN)]: *have a surprised expression for a second, then smile again*

... Me too.

*Then, lean my body against him a little*
[Model  (EN)]: *smile again, making a surprised look a bit* ... Me too. And lean a little against him*
--------------------------------------------------
[Input (KR)]: 비주술사라 주령을 볼 수 없다

...어... 뭐 하시는 거에요??

전혀 감을 못 잡는다
[Target (EN)]: *can't quite see cursed spirits since I'm a non-sorcerer*

... Uh... What are you doing?

*have no idea*
[Model  (EN)]: *surprised* ... Uh... What are you doing? I can't feel it at all*
--------------------------------------------------
[Input (KR)]: 하지만... 그거 좋았어...?

